In [1]:
%pip install bokeh

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import DatetimeTickFormatter, NumeralTickFormatter, FixedTicker
from bokeh.io import curdoc
import pandas as pd
import re
import numpy as np

In [3]:
with open('soal_chart_bokeh.txt', 'r') as file:
    lines = file.readlines()
    
timestamps = []
bitrates = []
transfers = []

for line in lines:
    timestamp_match = re.search(r'Timestamp: (\d+-\d+-\d+ \d+:\d+:\d+)', line)
    transfer_match = re.search(r'\[  5\] +\d+\.\d+-\d+\.\d+ +sec +(\d+\.\d+) +MBytes', line)
    bitrate_match = re.search(r'(\d+\.\d+) +Mbits/sec', line)
    
    if timestamp_match:
        timestamps.append(timestamp_match.group(1))
    if transfer_match:
        transfers.append(float(transfer_match.group(1)))
    if bitrate_match:
        bitrates.append(float(bitrate_match.group(1)))

min_length = min(len(timestamps), len(transfers), len(bitrates))
timestamps = timestamps[:min_length]
transfers = transfers[:min_length]
bitrates = bitrates[:min_length]

data = pd.DataFrame({
    'Timestamp': pd.to_datetime(timestamps),
    'Transfer (MBytes)': transfers,
    'Bitrate (Mbits/sec)': bitrates,
})

display(data)

,Timestamp,Transfer (MBytes),Bitrate (Mbits/sec)
0,2024-07-30 10:23:42,1.13,1.61
1,2024-07-30 11:23:53,2.79,1.75
2,2024-07-30 12:24:04,8.92,1.03
3,2024-07-30 13:24:15,11.80,1.55
4,2024-07-30 14:24:31,9.98,2.06
...,...,...,...
191,2024-08-08 03:15:42,1.68,94.40
192,2024-08-08 04:15:53,1.23,83.90
193,2024-08-08 05:16:04,1.29,83.90
194,2024-08-08 06:16:15,1.11,94.40


In [4]:
start_date = pd.Timestamp('2024-07-30')
end_date = pd.Timestamp('2024-08-02')
filtered_data = data[(data['Timestamp'] >= start_date) & (data['Timestamp'] < end_date)]

hourly_data = filtered_data.resample('H', on='Timestamp').mean()
hourly_data = hourly_data.interpolate(method='linear')

In [5]:
def get_custom_ticks(start, end, freq='6H'):
    ticks = pd.date_range(start=start, end=end, freq=freq)
    return ticks

tick_positions = get_custom_ticks(start_date, end_date)
tick_positions_ms = tick_positions.astype(np.int64) // 10**6

In [7]:
p = figure(x_axis_type='datetime', title="Testing jaringan", height=450, width=1000)

p.xaxis.formatter = DatetimeTickFormatter(
    hours="%m/%d/%Y\n%H:%M:%S",
    minutes="%m/%d/%Y\n%H:%M:%S",
    seconds="%m/%d/%Y\n%H:%M:%S"
)

p.yaxis.formatter = NumeralTickFormatter(
    format="0.00"
)

p.line(hourly_data.index, hourly_data['Bitrate (Mbits/sec)'], line_width=2, legend_label="Bitrate (Mbits/sec)", color='blue')
p.scatter(hourly_data.index, hourly_data['Bitrate (Mbits/sec)'], size=6, color='blue', fill_alpha=0.5)
p.xaxis.ticker = FixedTicker(ticks=tick_positions_ms)
p.yaxis.axis_label = 'Speed (Mbps)'
p.xaxis.axis_label = 'DATE TIME'

# curdoc().theme = "dark_minimal"
show(p)
output_notebook()


Loading BokehJS ...